This notebook corresponds to the cloud function: `update_all_sp_maturities`.

In [1]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../ahmad_creds.json'

In [7]:
import numpy as np
import pandas as pd
from google.cloud import bigquery
import requests as r
import datetime
from google.cloud import secretmanager
import smtplib
from email.mime.text import MIMEText

In [ ]:
TARGET_DATE = datetime.datetime(2023, 11, 3)

In [ ]:
date_to_rebuild_for = pd.to_datetime(TARGET_DATE).date()

In [3]:
PROJECT_ID = 'eng-reactor-287421'

# table names in Bigquery in `eng-reactor-287421.spBondIndexMaturities` group
TABLE_NAMES = ['sp_15plus_year_national_amt_free_index',
               'sp_12_22_year_national_amt_free_index',
               'sp_7_12_year_national_amt_free_index',
               'sp_high_quality_index',
               'sp_high_quality_intermediate_managed_amt_free_index',
               'sp_high_quality_short_index',
               'sp_high_quality_short_intermediate_index',
               'sp_long_term_national_amt_free_municipal_bond_index_yield']

# defining IDs using dictionaries so we don't need to rely on indexing for lists, etc, to remove any ambiguity; the INDEX_IDS are unique identifiers for each S&P index to request their data from the S&P API, these values were scraped directly from S&P
INDEX_IDS = {'sp_15plus_year_national_amt_free_index': 92346704, 
             'sp_12_22_year_national_amt_free_index': 946546, 
             'sp_7_12_year_national_amt_free_index': 946545, 
             'sp_high_quality_intermediate_managed_amt_free_index': 92404510, 
             'sp_high_quality_short_intermediate_index': 10001820, 
             'sp_high_quality_short_index': 10001819, 
             'sp_high_quality_index': 10001818, 
             'sp_long_term_national_amt_free_municipal_bond_index_yield': 946547}

Functions to email error message if update fails.

In [4]:
def access_secret_version(project_id, secret_id, version_id):
    client = secretmanager.SecretManagerServiceClient()    # create the Secret Manager client
    name = f'projects/{project_id}/secrets/{secret_id}/versions/{version_id}'    # build the resource name of the secret version
    response = client.access_secret_version(request={'name': name})    # access the secret version
    payload = response.payload.data.decode('UTF-8')
    return payload


def send_error_email(subject,error_message):
    sender_email = access_secret_version('eng-reactor-287421', 'notifications_username', 'latest')
    password = access_secret_version('eng-reactor-287421', 'notifications_password', 'latest')
    receiver_email = 'eng@ficc.ai'
    
    msg = MIMEText(error_message)
    msg['Subject'] = subject
    msg['From'] = sender_email
    msg['To'] = receiver_email

    smtp_server = 'smtp.gmail.com'
    port = 587
    sender_email = 'notifications@ficc.ai'

    with smtplib.SMTP(smtp_server, port) as server:
        try:
            server.starttls()
            server.login(sender_email, password)
            server.sendmail(sender_email, receiver_email, msg.as_string())
        except Exception as e:
            print(e)
        finally:
            server.quit() 

Functions to upload data to bigquery.

In [5]:
def get_schema():
    schema=[bigquery.SchemaField('effectivedate', bigquery.enums.SqlTypeNames.DATE),
            bigquery.SchemaField('weightedAverageMaturity', bigquery.enums.SqlTypeNames.FLOAT),
            bigquery.SchemaField('weightedAverageDuration', bigquery.enums.SqlTypeNames.FLOAT)]
    return schema


def upload_data(df, TABLE_ID):
    client = bigquery.Client(project=PROJECT_ID, location='US')
    job_config = bigquery.LoadJobConfig(schema=get_schema(), 
                                        write_disposition='WRITE_APPEND')
    job = client.load_table_from_dataframe(df, TABLE_ID, job_config=job_config)
    job.result()

In [15]:
def scrape_maturities(index_id):
    '''Scrape data from S&P website.'''
    # the following link was obtained from S&P's html code, it requests the data for a given index identified by the indexId variable
    action_url = f'https://www.spglobal.com/spdji/en/util/redesign/index-data/get-performance-data-for-datawidget-redesign.dot?indexId={index_id}&language_id=1&_=1627373317848'
    
    res = r.get(action_url)
    res_json = res.json()    # the API returns a json file which, among other things, contains the average maturity and duration of the index and the date those figures were last updated (the effectivedate)
    
    weighted_average_maturity = res_json['indexPerformanceHolder']['indexPerformance']['totalReturn'].get('weightedAverageMaturity')
    weighted_average_duration = res_json['indexPerformanceHolder']['indexPerformance']['totalReturn'].get('weightedAverageDuration')
    
    if not weighted_average_maturity:
        weighted_average_maturity = np.nan
    if not weighted_average_duration:
        weighted_average_duration = np.nan
    
    # return a dictionary which will become a single dataframe row, which is appended to the current bigquery table
    data = {'effectivedate': date_to_rebuild_for,
            'weightedAverageMaturity': weighted_average_maturity,
            'weightedAverageDuration': weighted_average_duration}
    
    return data

In [22]:
def main(args):
    for table in TABLE_NAMES:    # for each index, we scrape maturities and upload
        print(table)
        index_id = INDEX_IDS[table]
        try:
            data = scrape_maturities(index_id)
            data = pd.DataFrame(data, index=[0])    # save next entry as a single row dataframe and upload to bigquery
            upload_data(data, f'eng-reactor-287421.spBondIndexMaturities.{table}')
        except Exception as e:
            print('Cloud function error raised:', e)
            raise e    # this ensures that even after we catch an error and send an email alert, we raise the error to make the function fail. This allows the cloud function instance to fail and retry.

    return 'Run Success'

In [23]:
main('test')

sp_15plus_year_national_amt_free_index
sp_12_22_year_national_amt_free_index
sp_7_12_year_national_amt_free_index
sp_high_quality_index
sp_high_quality_intermediate_managed_amt_free_index
sp_high_quality_short_index
sp_high_quality_short_intermediate_index
sp_long_term_national_amt_free_municipal_bond_index_yield


'Run Success'